In [1]:
#NOTE: use paimg9 env
import sys
import os
import numpy as np
import openslide
import pandas as pd
import warnings
import torch
import torch.nn as nn

sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists, count_label, set_seed
from train_utils import ModelReadyData_diffdim, get_feature_label_array_dynamic_tma
warnings.filterwarnings("ignore")

In [2]:
####################################
######      USERINPUT       ########
####################################
SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
SELECTED_FEATURE = [str(i) for i in range(0,1024)] + ['TUMOR_PIXEL_PERC'] #If retccl 2048, if uni 1024
TUMOR_FRAC_THRES = 0.9
pixel_overlap = 0     
save_image_size = 250
cohort_name = "TAN_TMA_Cores"  #TAN_TMA_Cores
feature_extraction_method = 'uni1'
folder_name = cohort_name + "/" + "IMSIZE" + str(save_image_size) + "_OL" + str(pixel_overlap) + "/" 

##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
feature_path = proj_dir + 'intermediate_data/4_tile_feature/' + folder_name
tumor_info_path =  proj_dir + 'intermediate_data/2_cancer_detection/' + folder_name
################################################
#Create output dir
################################################
outdir =  os.path.join(proj_dir + 'intermediate_data/5_model_ready_data', 
                       folder_name, 
                       'feature_' + feature_extraction_method, 
                       'TFT' + str(TUMOR_FRAC_THRES))
create_dir_if_not_exists(outdir)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
set_seed(0)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/5_model_ready_data/TAN_TMA_Cores/IMSIZE250_OL0/feature_uni1/TFT0.0' created.
cuda:0


In [9]:
#All available IDs
tma_ids = [x.replace('.tif','') for x in os.listdir(feature_path) if x != '.DS_Store'] #677 
tma_ids.sort()

len(tma_ids)
#Check cancer detection
all_tile_info_list = list()
for pt in tma_ids:
    cur_tile_info = pd.read_hdf(feature_path + pt + '/features/' + 'features_alltiles_' +  feature_extraction_method + '.h5', key='tile_info')
    cur_tumor_info_df = pd.read_csv(os.path.join(tumor_info_path, pt, "ft_model/", pt + "_TILE_TUMOR_PERC.csv"))
    cur_tile_info = cur_tile_info.merge(cur_tumor_info_df, on = ['SAMPLE_ID', 'MAG_EXTRACT', 'SAVE_IMAGE_SIZE', 'PIXEL_OVERLAP',
                                                                 'LIMIT_BOUNDS', 'TILE_XY_INDEXES', 'TILE_COOR_ATLV0', 'WHITE_SPACE',
                                                                 'TISSUE_COVERAGE'])
    all_tile_info_list.append(cur_tile_info)

all_tile_info = pd.concat(all_tile_info_list)

all_tile_info_thres = all_tile_info.loc[all_tile_info['TUMOR_PIXEL_PERC'] >= TUMOR_FRAC_THRES]

nocancer_ids = [x for x in tma_ids if x not in list(set(all_tile_info_thres['SAMPLE_ID']))]
print("No Cancer IDs",nocancer_ids)

#Excluded 
toexclude_ids = nocancer_ids 

#Exclude ids in ft_train or processed
selected_ids = [x for x in tma_ids if x not in toexclude_ids] #199
print(len(selected_ids))

No Cancer IDs ['TMA97A-2-2', 'TMA97A-5-7', 'TMA97A-6-3', 'TMA97A-6-6', 'TMA97A-8-4', 'TMA97B-1-13', 'TMA97B-3-2', 'TMA97B-3-3', 'TMA97B-6-7', 'TMA97C-3-13', 'TMA97C-4-10', 'TMA97C-4-7', 'TMA97C-6-1', 'TMA97C-6-12', 'TMA97C-7-1', 'TMA97C-8-13', 'TMA97C-9-1', 'TMA97D-1-4', 'TMA97D-2-14', 'TMA97D-2-6', 'TMA97D-2-9', 'TMA97D-3-3', 'TMA97D-4-6', 'TMA97D-7-12', 'TMA97D-7-13', 'TMA97D-7-6', 'TMA97F-3-9', 'TMA97G-7-1', 'TMA97G-7-4', 'TMA97G-7-5', 'TMA97H-2-8', 'TMA97H-3-10', 'TMA97H-8-1', 'TMA97H-8-14', 'TMA97H-8-2', 'TMA97I-1-13', 'TMA97I-2-1', 'TMA97I-3-9', 'TMA97I-4-12', 'TMA97I-5-11', 'TMA97I-5-13', 'TMA97I-5-2', 'TMA97I-5-3', 'TMA97I-5-6', 'TMA97I-5-9', 'TMA97I-7-13', 'TMA97I-7-8']


In [11]:
#Exclude ids in ft_train or processed
toexclude_ids = nocancer_ids 
selected_ids = [x for x in tma_ids if x not in toexclude_ids] #199
print(len(selected_ids))

630


In [4]:
############################################################################################################
#Get features and labels
#NOTE: OPX_005 has no tumor tiles in fold0 train, so excluded in this step
############################################################################################################
feature_name = 'features_alltiles_' +  feature_extraction_method 
feature, label, info, tf_info, select_val_ids = get_feature_label_array_dynamic_tma(feature_path, 
                                                                                    tumor_info_path, 
                                                                                    feature_name, 
                                                                                    selected_ids,
                                                                                    SELECTED_LABEL, 
                                                                                    SELECTED_FEATURE, 
                                                                                    TUMOR_FRAC_THRES)

0
100


KeyboardInterrupt: 

In [ ]:
torch.save(feature,   outdir + '/tma_feature.pth')
torch.save(label,   outdir + '/tma_label.pth')
torch.save(info,   outdir + '/tma_info_.pth')
torch.save(tf_info,   outdir + '/tma_tfinfo.pth')
torch.save(select_val_ids,   outdir + '/tma_ids.pth')

In [ ]:
############################################################################################################
#Count Distribution
############################################################################################################
val_counts = count_label(label, SELECTED_LABEL, "TMA")
val_counts
val_counts.to_csv(outdir + '/tma_counts.csv')

In [ ]:
#Get model ready data
val_data = ModelReadyData_diffdim(feature,label,tf_info,include_tumor_fraction = False, include_cluster = False, feature_name =feature_extraction_method)
torch.save(val_data, outdir + '/tma_data.pth')